In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from tennis_tracker.download_data.extract_keypoints import read_json_file
from groundingdino.util.inference import load_image_quarters

def plot_quarter(box: list, image: np.array):
    plt.imshow(image)
    cx, cy, w, h = box
    image_height, image_width, _ = image.shape
    x1 = int((cx - w/2) * image_width)
    x2 = int((cx + w/2) * image_width)
    y1 = int((cy - h/2) * image_height)
    y2 = int((cy + h/2) * image_height) 
    plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, color='red', fill=False))
    plt.show()

FILE_PATH = (
    "/Users/derek/Desktop/tennis_tracker/tennis_tracker/ball_tracking/labels.json"
)
FPS = 30
FRAME_WIDTH, FRAME_HEIGHT = 640, 480
COURT_HEIGHT = 799
COURT_WIDTH = 560
VIDEO_PATH = "/Users/derek/Desktop/tennis_tracker/tennis_tracker/download_data/dataset/Tiafoe Takes On Evans; Khachanov& Mannarino Also In Action ｜ Almaty 2024 Highlights Day 4 [Q1iTjk444RU].webm"


def get_next_frame(data_entry: dict, court_image: np.array):
    """We want to get tracked points and draw them on the court image"""
    tracked_points = data_entry["ball_tracking_transformed_coords"]
    original_image = cv2.imread(data_entry["actual_path"])
    original_image = cv2.resize(original_image, (640, 480))
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    for point in tracked_points:
        cv2.circle(court_image, [int(p) for p in point], 10, (0, 0, 255), -1)
    return original_image, tracked_points


data = read_json_file(FILE_PATH)

tiafoe_video_frames = []
keys = list(data.keys())
for key in keys:
    if "Tiafoe" in key:
        tiafoe_video_frames.append((key, int(key.split("/")[-1].split(".")[0])))

tiafoe_video_frames.sort(key=lambda x: x[1])
tiafoe_video_frames = [x[0] for x in tiafoe_video_frames]
court_image = cv2.imread(
    "/Users/derek/Desktop/tennis_tracker/tennis_tracker/player_location/padded_court.jpg"
)

idx = 9
path = data[tiafoe_video_frames[idx]]['actual_path']
quarters = load_image_quarters(path)